<a href="https://colab.research.google.com/github/mavela/Linguistics-with-conllu-data/blob/master/Notebook_Poitiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Foreword

Focus here on ready-made Python scripts

(Although some of the first commands are in Bash)

Bash commands need to begin with` ! `here

All the commands can be run by clicking the arrow

You can also change input files and variables as instructed below
Focus here on ready-made Python scripts

**Questions** sections point you to possibly interesting things

All the scripts are downloadable at https://github.com/mavela/Linguistics-with-conllu-data.git (disclaimer for code beauty!)

# Part 1

#Preparations
Let's download the data from Github - this is done using `git clone + repo address`

`Git clones` the entire folder to the colab directory

`% cd` takes us to the correct directory

`! ls` lists the files in that directory

In [ ]:
! git clone https://github.com/mavela/Linguistics-with-conllu-data.git

In [ ]:
% cd Linguistics-with-conllu-data/
! ls

In [ ]:
! echo "Here is the data folder"
! ls data/


### Then let's have a look at the data

The files are from the Corpus of Online Register of English (Biber & Egbert 2018)

df = discussion forum

ds = description with intent to sell

sr = sports report

ht = how-to

You can print N first lines of the file with `head -N`

### Questions: 

- You can try to print different files!

In [ ]:
! head -20 data/sr.conllu

## Hoq many tokens?
### Questions

- How many tokens?
- Are there differences between the datasets?

In [ ]:
from analyze import count_words, most_frequent, extract_register, print_text

print("Total word count of the conllu file is", count_words("data/sr.conllu"), "tokens")

## Print words, lemmas or other things

`print_text` prints one text per line the text featurized as we indicate.

We can specify the kind of feature by referring to the column name in the Conll format.

The columns are:

`ID, FORM, LEMMA, UPOS, XPOS, FEAT, HEAD, DEPREL, DEPS, MISC`

The numeric argument indicates how many texts we want.

### Questions
You can try how the texts look like when featurized. How would you estimate the parser performance?

In [ ]:
print(print_text("data/df.conllu", "FORM", 2))
print()
## Then try with some other text class and feature!

## Part 2

##Type A perspective: Analyzing / searching for individual words / lemmas / something

### Questions

How many times a particular lemma appears in the corpus?

What are its most frequent dependency relations? (Or other tags - you can change the DEPREL / FORM part and the conllu file!)

In [ ]:
from analyze import count_specific_lemma, count_word_context

print(count_specific_lemma("you", "data/sr.conllu", "FORM"))
print(count_specific_lemma("think", "data/sr.conllu", "DEPREL"))

### Question
How does a word appear in context? 

In [ ]:
from analyze import read_conllu
ID,FORM,LEMMA,UPOS,XPOS,FEAT,HEAD,DEPREL,DEPS,MISC=range(10)

count = 0
for comm, sent in read_conllu(open("data/sr.conllu", "r")): # here you can change the data file
    if "nsubj" in [token[DEPREL].lower() for token in sent]: #here you can put any word form instead of "you"
        count += 1
        if count > 5: # here we specify how many sentences we want to see
          break
        else:
          print(" ".join(token[FORM] for token in sent)) # now this prints the sentence FORMs and DEPRELs, but these can be changed as well
          print(" ".join(token[DEPREL] for token in sent))
          print()


## Part 2

## Type B perspective: Text level
Most frequent tokens + lemmas in a text

In [ ]:
print("Most frequent tokens")
print(most_frequent("data/sr.conllu", "FORM", 10)) # the number defines how many we want to see

print("Most frequent lemmas")
print(most_frequent("data/sr.conllu", "LEMMA", 10)) # the number defines how many we want to see

Distribution of POS tags or dependency relations

The Conllu tagsets (columns) are defined as: `ID,FORM,LEMMA,UPOS,XPOS,FEAT,HEAD,DEPREL,DEPS,MISC`

In [ ]:
print("Most frequent frequent part-of-speech tags")
print(most_frequent("data/sr.conllu", "UPOS", 10))

print("Then the most frequent dependency relations")
print(most_frequent("data/sr.conllu", "DEPREL", 10))


### Question:
- Can you see any differences in the most frequent words between the files? Can you think of any reasons for the similarities / differences you see?

### Focusing text-level analysis to particular tags

### Questions

What are the most frequent adjectives / verbs / etc. in the dataset?

Do these differ between the datasets?

In [ ]:
print("Most frequent lemmas under a specific tagset(column).")
print("For instance, the most frequent lemmas that receive the ADJ tag in the UPOS column.")
print()
print(most_frequent("data/sr.conllu", "UPOS", 10, "ADJ"))

print("Or the most frequent lemmas that receive nsubj that in the DEPREL column")
print()
print(most_frequent("data/sr.conllu", "DEPREL", 10, "nsubj"))


# Keyness

### Standard target+reference corpus comparison with log likelihood

`"FORM"` specifies the conlllu format column you want to focus, so you can use `"LEMMA"` too

Note that we call the keyness script from bash here.

The keywords are counted for `ds.conllu` in comparison with `ht.conllu`

Remember that you can change these!

In [ ]:
! python3 keyness-conllu.py data/ds.conllu data/ht.conllu "FORM" | head -30 


### Questions: 
- What happens if you try to get keywords using lemmas? 
- Do all the registers get good keywords, or are some registers described better? Do you get descriptive keywords for all the registers?
- Does changing the reference corpus change the results?

### Note!
An essential part of any linguistic analysis is to understand what the numerically extracted results (here the keywords) mean in context. To this end, you  can print texts with the keywords using the `read_conllu.py` above.

## Text dispersion keyness

Counted the same way as the standard keyness above

In [ ]:
! python3 text_dispersion.py data/sr.conllu data/df.conllu "FORM" | head -40

### Questions:
- How do text dispersion keywords differ from the standard ones? Or do they?
- And again: is the keyword quality better for some registers? What happens if you try with lemmas?

## Filtering
So far, we have not used the benefits of the conllu format w.r.t the filtering of "uninformative" parts-of-speech.

However, this can be useful (depending on the keyness method we use). 

In the script below you can indicate the POS classes you want to exclude. The pos tags used in the conll format are:
```
ADJ: adjective
ADP: adposition
ADV: adverb
AUX: auxiliary
CCONJ: coordinating conjunction
DET: determiner
INTJ: interjection
NOUN: noun
NUM: numeral
PART: particle
PRON: pronoun
PROPN: proper noun
PUNCT: punctuation
SCONJ: subordinating conjunction
SYM: symbol
VERB: verb
X: other
```



The keyness scripts using filtering need a stop word file where you list all the POS classes that you want to filter out.

In [ ]:
! echo "ADJ PUNCT DET PRON PART" > stop.txt # echo prints the tags to the file
! cat stop.txt # cat prints the file here so we can have a look at it

You can try out the filtered keyness scripts below. 

Both keyness scripts take `stop.txt` as an argument, as you can notice from the command

In [ ]:
! python3 keyness-filt.py data/sr.conllu data/df.conllu "FORM" stop.txt | head -40

In [ ]:
! python3 text_dispersion_filt.py data/sr.conllu data/df.conllu FORM stop.txt | head -40

### Questions:
Do the results become better when you use filtering? 

Are there differences between the methods?

What POS classes would you see as the best to use as stop?

Do you see any risks in using such stop lists?

What kinds of research questions could you use these methods to?